<a href="https://colab.research.google.com/github/IslandicMoon/Home_Sales/blob/main/Home_Sales_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.4.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:6 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,202 kB]
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [1,430 kB]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:11 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [1,404 kB]
Hit:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:13 http://security.ubuntu.com/ubuntu ja

Exception: ignored

In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

ModuleNotFoundError: ignored

In [3]:
# 1. Read in the AWS S3 bucket into a DataFrame.
  from pyspark import SparkFiles
  url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
  spark.sparkContext.addFile(url)



IndentationError: ignored

In [4]:
# 2. Create a temporary view of the DataFrame.
home_sales = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)

home_sales.createOrReplaceTempView("home_sales_revised")

NameError: ignored

In [5]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?

avg_price_4_bedroom = home_sales.filter(home_sales["bedrooms"] == 4).groupBy("year_sold").avg("price")
avg_price_4_bedroom.show()

NameError: ignored

In [6]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?

avg_price_3_bed_3_bath = home_sales.filter((home_sales["bedrooms"] == 3) & (home_sales["bathrooms"] == 3)).groupBy("year_built").avg("price")
avg_price_3_bed_3_bath.show()

NameError: ignored

In [7]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
avg_price_filtered = home_sales.filter(
    (home_sales["bedrooms"] == 3)
    & (home_sales["bathrooms"] == 3)
    & (home_sales["floors"] == 2)
    & (home_sales["sqft"] >= 2000)
).groupBy("year_built").avg("price")
avg_price_filtered.show()


NameError: ignored

In [8]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()

avg_price_over_350k = home_sales.filter(home_sales["price"] >= 350000).groupBy().avg("view")
avg_price_over_350k.show()

print("--- %s seconds ---" % (time.time() - start_time))

NameError: ignored

In [9]:
# 7. Cache the the temporary table home_sales.
home_sales.createOrReplaceTempView("home_sales")

NameError: ignored

In [10]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales_revised')

NameError: ignored

In [11]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()

avg_price_over_350k_cached = spark.sql("SELECT AVG(view) FROM home_sales WHERE price >= 350000")
avg_price_over_350k_cached.show()

print("--- %s seconds ---" % (time.time() - start_time))


NameError: ignored

In [ ]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
home_sales.write.partitionBy("date_built").parquet("/content/home_sales_parquet")

In [ ]:
# 11. Read the parquet formatted data.
parquet_data = spark.read.parquet("/content/home_sales_parquet")

In [ ]:
# 12. Create a temporary table for the parquet data.
parquet_data.createOrReplaceTempView("parquet_data")

In [ ]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.

start_time = time.time()

avg_price_over_350k_parquet = spark.sql("SELECT AVG(view) FROM parquet_data WHERE price >= 350000")
avg_price_over_350k_parquet.show()

print("--- %s seconds ---" % (time.time() - start_time))

--- 7.104873657226562e-05 seconds ---


In [ ]:
# 14. Uncache the home_sales temporary table.
spark.catalog.uncacheTable("home_sales")

In [ ]:
# 15. Check if the home_sales is no longer cached

print(spark.catalog.isCached('home_sales'))